# A 10 minute `swast_forecast` tutorial

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')


#arima
from statsmodels.tsa.arima.model import ARIMA

#prophet
from fbprophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
class FbProphetWrapper(object):
    '''
    Wrapper for fbprophet.Prophet.
    
    This is used to give ARIMA and Prophet objects the same interface.
    
    Attributes:
    --------
    .fittedvalues
    .resid
    
    Methods:
    --------
    .fit(train)
    .predict(h, return_conf_int=False, alpha=0.2)

    '''
    def __init__(self, training_index, holidays=None, interval_width=0.8,
                 mcmc_samples=0, changepoint_prior_scale=0.05):
        '''
        
        Params:
        ------
        training_index: pd.DatetimeIndex
            Dates in the training datas
        
        holidays: array-like
            Dates of holidays leading into the future
            
        interval_width: float
            Width of confidence interval
            
        mcmc_samples: int
            number of MCMC samples to takes to assess uncertainty
            in slope
        
        '''
        self._training_index = training_index
        self._holidays = holidays
        self._interval_width = interval_width
        self._mcmc_samples = mcmc_samples
        self._cp_prior_scale = changepoint_prior_scale

    def _get_resids(self):
        return self._train - self._forecast['yhat'][:-self._h]

    def _get_preds(self):
        return self._forecast['yhat'][:-self._h].to_numpy()

    def fit(self, train):
        
        self._model = Prophet(holidays=self._holidays, 
                              interval_width=self._interval_width,
                              mcmc_samples=self._mcmc_samples,
                              changepoint_prior_scale=self._cp_prior_scale,
                              daily_seasonality=False)
        
        
        self._model.fit(self._pre_process_training(train))
        self._t = len(train)
        self._train = train
        self.predict(len(train))

    def _pre_process_training(self, train):

        if len(train.shape) > 1:
            y_train = train[:, 0]
        else:
            y_train = train

        y_train = np.asarray(y_train)
            
        #hack!!
        if len(y_train) > len(self._training_index):
            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
        
        
        prophet_train = pd.DataFrame(self._training_index)
        prophet_train['y'] = y_train
        prophet_train.columns = ['ds', 'y']
        
        return prophet_train

    def predict(self, h, return_conf_int=False, alpha=0.2):
        '''
        Predict h periods ahead
        
        Params:
        ------
        h: int
            time periods ahead to predict
            
        return_conf_int: bool, optional (default=False)
            return confidence intervals as well as point forecasts
            
        alpha: float, optional (default=0.2)
            return prophet 1 - alpha prediction interval.  Only works
            if return_conf_int = True
        '''

        #this is because prophit requires a refit if alpha is different
        if (1 - alpha) != self.interval_width:
            self.interval_width = 1 - alpha
            #refit model to get new predi interval
            self.fit(self.train)
        
        if isinstance(h, (np.ndarray, pd.DataFrame)):
            h = len(h)
            
        self._h = h
        future = self._model.make_future_dataframe(periods=h)
        self._forecast = self._model.predict(future)

        if return_conf_int:
            return (self._forecast['yhat'][-h:].to_numpy(), 
                    self._forecast[['yhat_lower', 'yhat_upper']][-h:].to_numpy())
        else:
            return self._forecast['yhat'][-h:].to_numpy()
            

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)